<a href="https://colab.research.google.com/github/patriciamediavilla/L1P2/blob/main/PFG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PROYECTO FIN DE GRADO**

Patricia Mediavilla

Este Proyecto Fin de Grado tiene como objetivo emplear técnicas de Machine Learning con el fin de conseguir una clasificación de los datos. Se verán distintos algoritmos de clasificación clásica centrándose en el algoritmo SVM, Support Vector Machine.  La clasificación en este proyecto no va a ser únicamente clásica, se introducirán conceptos cuánticos con el fin de implementar un algoritmo de clasificación de QSVM, Quantum Support Vector Machine. Se verá en profundidad cómo funciona este algoritmo y por último se combinan los dos algoritmos, el cásico y el cuántico para mejorar la precisión del modelo.

Esto se va a plicar a un caso de uso en relación con la empresa, en concreto se pordrá ver si un cliente se va a ir o no de la empresa antes de que suceda, esto de llama **churn** que por definición es una métrica que mide la cantidad de cleintes que han dejado de seguir a una empresa

id contact id

activity_new category of the company's activity

campaign_disc_ele code of the electricity campaign the customer last subscribed to

channel_sales code of the sales channel

cons_12m electricity consumption of the past 12 months

cons_gas_12m gas consumption of the past 12 months

cons_last_month electricity consumption of the last month

date_activ date of activation of the contract

date_end registered date of the end of the contract

date_first_activ date of the first contract of the client

date_modif_prod date of last modification of the product

date_renewal date of the next contract renewal

forecast_base_bill_ele forecasted electricity bill baseline for next month

forecast_base_bill_year forecasted electricity bill baseline for calendar year

forecast_bill_12m forecasted electricity bill baseline for 12 months

forecast_cons forecasted electricity consumption for next month

forecast_cons_12m forecasted electricity consumption for next 12 months

forecast_cons_year forecasted electricity consumption for next calendar year

forecast_discount_energy forecasted value of currecurrent discount

forecast_meter_rent_12m forecasted bill of meter rental for the next 12 months

forecast_price_energy_p1 forecasted energy price for 1st period

forecast_price_energy_p2 forecasted energy price for 2nd period

forecast_price_pow_p1 forecasted power price for 1st period

has_gas indicated if clieclient is also a gas client

imp_cons current paid consumption

margin_gross_pow_ele gross margin on power subscription

margin_net_pow_ele net margin on power subscription

nb_prod_act number of active products and services

net_margin total net margin

num_years_antig antiquity of the client (in number of years)

origin_up code of the electricity campaign the customer first subscribed to

pow_max subscribed power

price_date reference date

price_p1_var price of energy for the 1st period

price_p2_var price of energy for the 2nd period

price_p3_var price of energy for the 3rd period

price_p1_fix price of power for the 1st period

price_p2_fix price of power for the 2nd period

price_p3_fix price of power for the 3rd period

churned has the client churned over the next 3 months